# GrEx1

# Predict 420 Sec 55

# Rich Rouse

## This assignment will follow the professor's instructions for GrEx1 as posted in Canvas

### Adjust the working directories to assist with locating files while looping

In [3]:
import os #import os module

In [4]:
os.getcwd()  #check the current working directory

'C:\\Users\\Rich\\Predict 401'

In [5]:
os.chdir("C:/Users/Rich/Documents/Northwestern/Predict 420/GrEx1") # target confirmed, change the directory

In [6]:
os.getcwd() #double-check that the working directory was successfully changed

'C:\\Users\\Rich\\Documents\\Northwestern\\Predict 420\\GrEx1'

In [7]:
#importing pandas and numpy
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

## Create the columns to be used for the DataFrames

In [8]:
cols = ["Airport_cols","Airline_cols","Routes_cols"]  #file names
things = [] # will become container for column names to be used in DataFrames
for i in cols:
   file = open("%s.txt"%i,"r")   #create a file object using open and the filename
   heading = file.read().split(", ")  #read the file object into a list after splitting the contents by the ","
   things.append(heading)  #add the newly created column name list to the container list "cols"

## Import the openair.org .dat files

In [9]:
frames = ["airlines","airports","Routes"] # file names
for i in frames: # generate a loop to open the .dat files as DataFrames
    globals()[i] = pd.read_csv("%s.dat"%i,header=None)  # create DataFrames from the .dat files using the read_csv() pandas function 

In [10]:
# check the frames were created successfully

## Assign the column names to the DataFrames

In [11]:
airports.columns = things[0]  #assign the column names by referencing the appropriate index in the container created above

In [12]:
airlines.columns = things[1]

In [13]:
Routes.columns = things[2]

## Conduct a merge between the Routes and Airports DataFrame for use in calculating flight  distances

In [14]:
##creating a condensed version of the Airports DataFrame that contains only lat and lon
locs = airports[["Airport_ID","Latitude","Longitude"]]
locs.head()

,Airport_ID,Latitude,Longitude
0,1,-6.081690,145.391998
1,2,-5.207080,145.789001
2,3,-5.826790,144.296005
3,4,-6.569803,146.725977
4,5,-9.443380,147.220001


In [15]:
print type(locs.ix[0,0])  #check the datatype of the Airport ID series in the locations DataFrame
print type(Routes.ix[0,3])  #check the datatype of the Airport ID series in the Routes DataFrame


<type 'numpy.float64'>
<type 'str'>


In [16]:
## so there is a mismatch in how the Airport IDs were interpeted during the import of the .dat files.  
## The datatypes must match in order to conduct the merge.

In [17]:
#convert Airport ID in locs to string to match the Routes Airport ID string format
locs["Airport_ID"] = locs["Airport_ID"].astype(str)

C:\Users\Rich\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [18]:
## merge the Routes DataFrame with the locs data
## merge on source airport ID's
result = pd.merge(Routes,locs, left_on="Source_ID", right_on = "Airport_ID")

In [46]:
## merge on destination airport ID's
result2 = pd.merge(result,locs, left_on="Destination_ID", right_on = "Airport_ID")

In [47]:
"""
define the haversine formula to calculate route distances distance between 
airport lats and long using the Route information
"""

def haversine(x):
    lat1,lon1,lat2,lon2 = x  #unpack ze values
    lat1 = np.deg2rad(lat1) #convert initial lat in degrees to radians
    lon1 = np.deg2rad(lon1) #convert initial lon in degrees to radians
    lat2 = np.deg2rad(lat2) #convert dest lat in degrees to radians
    lon2 = np.deg2rad(lon2) #convert dest lon in degrees to radians
    R = 6371 # earth mean radius in km
       
    deltalat = lat2 - lat1 #difference in initial and dest lat
    deltalon = lon2 - lon1 #difference in initial and dest lon
    
    #Apply Haversine formula
    a=np.sin(deltalat/2)**2+np.cos(lat1)*np.cos(lat2)*np.sin(deltalon/2)**2
    c=2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
    d=R*c
    return d

In [48]:
## Apply the Haversine function to the DataFrame to return distances
result2["Distance in KM"] = result2[["Latitude_x","Longitude_x","Latitude_y","Longitude_y"]].apply(haversine,axis=1)

In [49]:
## Convert the distances to integers; the default display was scientific notation
result2["Distance in KM"] = result2["Distance in KM"].astype(int)

## Check for duplicate entries in the DataFrames

In [23]:
## Check for duplicate entries within the dataframes using the DataFrame.duplicated() function.  Returns boolean True or False 
## if duplicate is detected
print sum(Routes.duplicated())  # use sum in conjunction with the .duplicated() function to count the True values

print sum(airlines.duplicated())

print sum(airports.duplicated())

0
0
0


In [24]:
# no duplicates detected

## Gather info on the datatypes and size of the DataFrames

In [25]:
##Gather info on the Datatypes and size of the DataFrames using the .info() function
print Routes.info() 

print airlines.info()

print airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67663 entries, 0 to 67662
Data columns (total 9 columns):
Airline           67663 non-null object
ID                67663 non-null object
Source            67663 non-null object
Source_ID         67663 non-null object
Destination       67663 non-null object
Destination_ID    67663 non-null object
Codeshare         14597 non-null object
Stops             67663 non-null int64
Equipment         67645 non-null object
dtypes: int64(1), object(8)
memory usage: 4.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6162 entries, 0 to 6161
Data columns (total 8 columns):
ID          6162 non-null int64
Name        6162 non-null object
Alias       5657 non-null object
IATA        1535 non-null object
ICAO        6075 non-null object
Callsign    5354 non-null object
Country     6147 non-null object
Active      6162 non-null object
dtypes: int64(1), object(7)
memory usage: 385.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71

## Determine defunct airlines

### I utilized the "Active" series in the Airlines dataframe to determine if the airline was defunct.  Using a boolean index/slice command on the "Active" series where the values == "N" was the best way I saw to proceed

In [26]:
##determine defunct airlines by performing a boolean check on the "Active" column in the airlines dataframe

len(airlines[airlines["Active"]=="N"]) #use len() function to count the length of the inactive airline frame

4906

## Determine flights from nowhere

### Flights from nowhere were defined as those having no "Source Airport ID" in the Routes information

In [27]:
##Determine "Flights from Nowhere" that do not originate from an airport in the airports data

print type(airports.ix[0,0])  # check the datatype of the Airport ID in the airports dataframe

print type(Routes.ix[0,3]) # check the datatype of the Source Airport ID in the Routes dataframe

<type 'numpy.int64'>
<type 'str'>


In [28]:
#Datatype mismatch between Airport ID in airports dataframe (integer) and Source ID in Routes (string)

In [29]:
comparitor = airports["Airport_ID"]  #create a new Series containing only Airport ID from Airport DataFrame
comparitor = comparitor.astype(str) #convert the series to a string datatype
type(comparitor[0]) #check type


str

### Time to find those "Flights from Nowhere".  Use the .inin() function in conjunction with the Source Airport ID Series from the Routes Frame.  The .isin() function will check the Source Airport ID Series against the comparitor Series that was created above

In [30]:
ghost_flights = Routes[Routes["Source_ID"].isin(comparitor)==False]
ghost_flights.head()

,Airline,ID,Source,Source_ID,Destination,Destination_ID,Codeshare,Stops,Equipment
38,2B,410,TGK,\N,DME,4029,NaN,0,CR2
54,2G,1654,KCK,\N,IKT,2937,NaN,0,AN4
175,2O,146,AOS,7167,KZB,7176,NaN,0,BNI
176,2O,146,KKB,7168,SYB,7173,NaN,0,BNI
178,2O,146,KOZ,7164,ORI,7165,NaN,0,BNI


In [31]:
len(ghost_flights)  #print the length of the ghost_flights DataFrame

859

## Determine the closest airport to my apartment using the haversine function defined earlier in the script

In [32]:
#find and define my latitude and longitude in Houston, Texas.
my_lat = 29.782677
my_lon = -95.44

In [33]:
#Create Series in the Airports dataframe containing my_lat and my_long as the entries
airports["my_lat"] = my_lat  
airports["my_lon"] = my_lon

In [34]:
#create a calculated column in the airports dataframe by applying the haversine formula to the Airport lat, airport long, my_lat, and my_long
airports["Proximity"] = airports[["Latitude","Longitude","my_lat","my_lon"]].apply(haversine, axis = 1)

In [35]:
sorted = airports.sort_values("Proximity", ascending=True).head()  # sort results by proximity to my apartment closest first

In [36]:
sorted

,Airport_ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz,Type,Source,my_lat,my_lon,Proximity
6435,9225,Andrau Airpark,Houston,United States,AAP,KAAP,29.722500,-95.588303,79,-6,A,America/Chicago,airport,OurAirports,29.782677,-95.44,15.803211
3351,3566,William P Hobby Airport,Houston,United States,HOU,KHOU,29.645399,-95.278900,46,-6,A,America/Chicago,airport,OurAirports,29.782677,-95.44,21.795887
3965,4226,West Houston Airport,Houston,United States,IWS,KIWS,29.818199,-95.672600,111,-6,A,America/Chicago,airport,OurAirports,29.782677,-95.44,22.788632
3335,3550,George Bush Intercontinental Houston Airport,Houston,United States,IAH,KIAH,29.984400,-95.341400,97,-6,A,America/Chicago,airport,OurAirports,29.782677,-95.44,24.361743
3959,4220,Sugar Land Regional Airport,Sugar Land,United States,SGR,KSGR,29.622299,-95.656502,82,-6,A,America/Chicago,airport,OurAirports,29.782677,-95.44,27.482450


### Andrau Airpark is the closest to me, although I have never heard of it before.  It is certainly not a substantial airfport compared with George Bush Intercontinental and Hobby.


[Andrau_Airpark](https://en.wikipedia.org/wiki/Andrau_Airpark)
<-------- this link shows the airport was only in use through 1998, which is why I have never heard of it before.

In [54]:
#Check the Andrau Airpark airport id against the Routes dataframe "source_id" Series
Andrau = str(9225)

type(Routes.ix[0,3])

Routes[Routes["Source"]=="AAP"]

,Airline,ID,Source,Source_ID,Destination,Destination_ID,Codeshare,Stops,Equipment


In [38]:
#did not find any departing flights from Andrau listed

In [39]:
# moved to Hobby as a fairly substantial airport expecting hits
Routes[Routes["Source"]=="HOU"].head()   #conduct Boolean check on the Source series within the DataFrame and return True values

,Airline,ID,Source,Source_ID,Destination,Destination_ID,Codeshare,Stops,Equipment
5530,AA,24,HOU,3566,DFW,3670,NaN,0,M83 M80
13397,AZ,596,HOU,3566,ATL,3682,Y,0,M88
14032,B6,3029,HOU,3566,BOS,3448,NaN,0,E90
14033,B6,3029,HOU,3566,JFK,3797,NaN,0,E90
20842,DL,2009,HOU,3566,ATL,3682,NaN,0,M88


In [40]:
# how many flights departing from hobby?
len(Routes[Routes["Source"]=="HOU"])

79

In [50]:
#Found 79 flights departing from hobby 

## How many flights entering Hobby from EGO Source Airport?

In [42]:
Routes[(Routes["Source"]=="EGO")&(Routes["Destination"]=="HOU")]  #dual condition boolean check on the Source Airport and the Destination

,Airline,ID,Source,Source_ID,Destination,Destination_ID,Codeshare,Stops,Equipment


### No flights located that departed EGO and arrived HOU.  How many flights departed EGO in total?

In [43]:
Routes[Routes["Source"]=="EGO"]  # boolean slice of the dataframe using the "EGO" Source

,Airline,ID,Source,Source_ID,Destination,Destination_ID,Codeshare,Stops,Equipment
9,2B,410,EGO,6156,KGD,2952,NaN,0,CR2
10,2B,410,EGO,6156,KZN,2990,NaN,0,CR2
572,3R,3545,EGO,6156,DME,4029,NaN,0,SU9
2901,7R,1469,EGO,6156,DME,4029,NaN,0,CR2
2902,7R,1469,EGO,6156,KUF,4118,NaN,0,CR2
2903,7R,1469,EGO,6156,ROV,2964,NaN,0,CR2
37314,KR,1844,EGO,6156,FRU,2912,NaN,0,320
47766,R3,1287,EGO,6156,KRR,2960,NaN,0,DH4
65901,Y7,13088,EGO,6156,GOJ,4274,NaN,0,738
66230,YQ,16882,EGO,6156,DME,4029,NaN,0,SFB


In [44]:
print len(Routes[Routes["Source"]=="EGO"])

11
